# Parameter definition
Choose all the settings for the database creation.

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset_chokepoint' #'colab_main_dataset'
input_dir = os.path.join(home, 'lr_video_project', 'image_datasets')
output_dir = os.path.join(home, 'lr_video_project', 'sql_database')
database_names=['enfsi2015'] # 'lfw', 'scface', 'forenface', 'enfsi', 'enfsi2015' 'xqlfw', 'utkface','chokepoint'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[13:29:02] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[13:29:02] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[13:29:03] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[13:29:03] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[13:29:03] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by pre

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2022-12-06 13:29:03.733037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 13:29:04.114135: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 13:29:05.227856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/cv2/../../lib64:
2022-12-06 13:29:05.228143: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'test_video_dataset_chokepoint',
 'input_dir': '/home/jmacarulla/lr_video_project/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/lr_video_project/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f9806461dd0>,
 'databases': [<sql_face.databases.Enfsi2015 at 0x7f980649e9d0>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
database.create_tables(serfiq=serfiq)

AssertionError: No such polymorphic_identity 'CPframe' is defined

In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)